In [ ]:
import pandas as pd
from nltk import word_tokenize
import nltk
from nltk.util import ngrams
from collections import Counter
import numpy as np
from fuzzywuzzy import fuzz
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_colwidth', 100000)
pd.set_option('display.max_rows', 500)
import string

In [ ]:
%reload_kedro

In [5]:
df = context.io.load('KOF_04152019_05')

2020-01-28 20:39:27,564 - kedro.io.data_catalog - INFO - Loading data from `KOF_04152019_05` (CSVLocalDataSet)...


In [7]:
df.head(1)

,code_style,name,brand,date,retail_price,colorway,story,kof_wants,avg_resale_stockx,silhouette,price_diff,net_profit,profitable,brand_code,black,white,brown,red,blue,yellow,orange,green,purple,multi_color,main_color,main_color_id,womens,bcollab,og,sp,qs,sb,ls,nrg,prm,nsw,retro,se,pe,gs,hs
0,314996-101,Nike Air Foamposite One Snakeskin,Nike,2019-04-15,220,Sail/Black-Habanero Red-Black,"Equipped with an eye-catching upper, this Air Foamposite One comes covered in a black and sail snakeskin textured material with black leather detailing on the eyestay that wraps all around the ankle. Additional black detailing can be seen on the tongue, laces, and inner liner while Habanero Red lands on the varios branding throughout the shoe, running down the tongue, on the pull tabs, and lastly on the rubber outsole.",2281,305,Air Foamposite One,85,49.545275,1,6,0.864923,0.796686,0.722427,0.843831,0.758602,0.727155,0.705313,0.68003,0.700628,0.541327,black,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
len(df)

## TF-IDF on sneaker names

In [8]:
names = list(df.name)

In [9]:
len(names)

6338

In [10]:
def tuple2string(tup): 
    str =  ' '.join(tup) 
    return str

In [11]:
grams = {}
for i in range(len(names)):
    all_grams = []
    for j in range(3,7):
        shoe = names[i]
        shoe = shoe.replace('nike ','')
        shoe = shoe.replace(' x ', '')
        shoe = shoe.replace('adidas', '')
        shoe = shoe.replace('og', '')
        shoe = shoe.replace('wmns', '')
        shoe = shoe.replace('high', '')
        shoe = shoe.replace('low', '')
        shoe = shoe.replace('gs', '')
        shoe = shoe.replace('mid', '')
        shoe = shoe.replace('retro', '')
        shoe = shoe.replace('footwear', '')
        shoe = shoe.replace('black', '')
        shoe = shoe.replace('white', '')
        shoe = shoe.translate(str.maketrans('','',string.punctuation))
        
        
        ngrams_raw = list(ngrams(shoe.split(), j))
        ngrams_clean = [tuple2string(ngrams_raw[i]) for i in range(len(ngrams_raw))]
        all_grams.extend(ngrams_clean)
    grams[names[i]] = all_grams

In [12]:
score_grams = []
for keys in grams:
    for j in range(len(grams[keys])):
        score = fuzz.ratio(keys, grams[keys][j])
        score_grams.append([keys, grams[keys][j], score])

In [13]:
silhoutte = pd.DataFrame(score_grams, columns = ['name', 'ngram', 'score'])

In [14]:
silhoutte

,name,ngram,score
0,Nike Air Foamposite One Snakeskin,Nike Air Foamposite,73
1,Nike Air Foamposite One Snakeskin,Air Foamposite One,71
2,Nike Air Foamposite One Snakeskin,Foamposite One Snakeskin,84
3,Nike Air Foamposite One Snakeskin,Nike Air Foamposite One,82
4,Nike Air Foamposite One Snakeskin,Air Foamposite One Snakeskin,92
...,...,...,...
61808,Air Jordan 11 Concord OG (1995),Jordan 11 Concord OG,78
61809,Air Jordan 11 Concord OG (1995),11 Concord OG 1995,73
61810,Air Jordan 11 Concord OG (1995),Air Jordan 11 Concord OG,87
61811,Air Jordan 11 Concord OG (1995),Jordan 11 Concord OG 1995,89


In [15]:
avg_ngram_scores = silhoutte.groupby(['ngram'])['score'].mean().reset_index(name='avg_score')
num_ngrams = silhoutte.groupby(['ngram'])['score'].count().reset_index(name='total')

In [16]:
silhoutte = pd.merge(silhoutte, num_ngrams, how='left', on='ngram')

In [17]:
silhoutte = pd.merge(silhoutte, avg_ngram_scores, how='left', on='ngram')

In [ ]:
silhoutte = silhoutte[silhoutte['total'] > 1]
silhoutte = silhoutte.drop(columns=['score'])

In [ ]:
silhoutte = silhoutte[silhoutte.avg_score<silhoutte.groupby('name').avg_score.transform('quantile',0.95)]
silhoutte = silhoutte[silhoutte.avg_score>silhoutte.groupby('name').avg_score.transform('quantile',0.01)]

In [ ]:
# silhoutte = silhoutte[silhoutte.total<silhoutte.groupby('name').total.transform('quantile',0.99)]
# silhoutte = silhoutte[silhoutte.total>silhoutte.groupby('name').total.transform('quantile',0.01)]

In [ ]:
silhoutte = silhoutte.sort_values(['total', 'avg_score'], ascending=False).drop_duplicates(['name'])


In [ ]:
silhoutte.head(500)

In [ ]:
silhoutte = silhoutte.rename(columns={'ngram':'silhoutte'})
merge_silhoutte = silhoutte[['name','silhoutte', 'avg_score']]

In [ ]:
df_silhouttes = pd.merge(df, merge_silhoutte, how='left', on='name')

In [ ]:
df_silhouttes.to_csv('/Users/nikhilbhargava/Documents/sneakairs/data/07_model_output/silhoutte-active-learning.csv', index=False)